# MlOps Notes

1. [Module 1: Introduction](#module1)

    1.1 [Introduction to MLOps](#intro)
    
    1.2 [Environment preparation](#environment)
    
      - 1.2.1 [VM Instance in GCP](#gcp)
      
      - 1.2.1 [Install Dependencies](#dependencies)
      
    1.3 [Course Overview](#overview)
    
    1.4 [MLOps Maturity Model](#maturity)

2. [Module 2: Experiment tracking and model management](#module2)
3. [Module 3: Orchestration and ML Pipelines](#module3)

    3.1 [Negative engineering and workflow orchestration](#negative-and-orchestration)
    
    3.2 [Introduction to Prefect 2.0](#prefect)

    3.3 [First Prefect flow and basics](#prefect-flow)
    
    - 3.3.1 [Adding a  Prefect flow](#flow)
    
    - 3.3.2 [Adding a Prefect task](#task)
    
    - 3.3.3 [Sequential Task Running](#seq)
    
    - 3.3.4 [Logs in Prefect](#logs)
    
    - 3.3.5 [Prefect UI](#ui)
    
    - 3.3.6 [Parameter Type Validation](#val)
    
    3.4 [Remote Prefect Orion Deployment](#remote)

    - 3.4.1 [Remote Prefect Orion](#orion)
    
    - 3.4.2 [Using Prefect Cloud](#cloud)
    
    - 3.4.3 [Defining Storage for Prefect localy](#storage-localy)
    
    - 3.4.4 [Defining Storage for Prefect in AWS S3](#storage-aws)

    3.5 [Deployment of Prefect flow](#deploy-prefect)
    
    - 3.5.1 [Deployment](#deploy)
    
    - 3.5.2 [CronSchedule](#schedule)
    
    - 3.5.3 [Work Queues](#queue)
    
    - 3.5.4 [Adding a Local Agent](#agent)
    

12. [References](#references)

This notes are about [MLOps Zoomcamp](https://github.com/DataTalksClub/mlops-zoomcamp)

# Module 1: Introduction <a name="module1"></a>

[Source](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/01-intro)

## Introduction to MLOps <a name="intro"></a>

[Video source](https://youtu.be/s0uaFZSzwfI?list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK)

***MLOps*** is a _set of best practices_ for bringing Machine Learning to production.

Machine Learning projects can be simpplified to just 3 steps:

1. ***Design*** - is ML the right tool for solving our problem?
   * _We want to predict the duration of a taxi trip. Do we need to use ML or can we used a simpler rule-based model?_
2. ***Train*** - if we do need ML, then we train and evaluate the best model.
3. ***Operate*** - model deployment, management and monitoring.

MLOps is helpful in all 3 stages.

[Back to the top](#)


## Environment preparation <a name="environment"></a>

[Video source](https://www.youtube.com/watch?v=IXSiYkP23zo&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK)


You may check the link above to watch the video in order to learn how to set up a Linux VM instance in Amazon Web Services.
You can prepare your environment in your local machine too, but in our case we are going to set up a VM instance in GCP.


### VM Instance in GCP <a name="gcp"></a>

Before to create an instance in GCP we need to generate a SSH key(if you want to know what SSH is you can check this [video](https://www.youtube.com/watch?v=RMS5zBYQIqA)). In your local console(in my case in Git Bash in windows) follow:

You can watch this [video](https://www.youtube.com/watch?v=ae-CV2KfoN0) or follow the next instructions.

1. **Create SSH keys:**

    Create(if you don't have) and go to ```~/.ssh``` directory and type:

```bash
ssh-keygen -t rsa -f KEY_FILENAME -C USER -b 2048
```
    Example:

```bash
ssh-keygen -t rsa -f gcp_ssh -C w10 -b 2048
```

[Source](https://cloud.google.com/compute/docs/connect/create-ssh-keys)

2. **Put this SSH key in GCP:**
    
    1. Copy public key:
    
        ```bash
        cat KEY_FILENAME.pub
        ```
        Example:
        ```bash
        cat gcp_ssh.pub
        ```
    2. In Cloud console go to **Metadata** -> **EDIT** -> **SSH keys** -> **Add item** -> **Paste public key** -> **Save**
    
_[Source](https://cloud.google.com/compute/docs/connect/add-ssh-keys)_
    
3. **Create VM Instance:**

    In Cloud console go to **Compute Engine** -> **VM Instances** -> **Create Instance** config the VM as:
    * name: `mlops-zoomcamp-vm`
    * region: `us-west4 (Las Vegas)`, zone: `us-west4-b`
    * serie: `E2`, type: `e2-standard-4`
    * boot disk image: `Ubuntu 22.04 LTS` boot disk type: `balanced persistent disk` size(gb): `30`
    
4. **Connect to VM Instance:** 

    Go to ```~./.ssh``` directory and locate the ```config``` type ```nano ~/.ssh/config```copy and paste:

```bash
Host mlops-zoomcamp-vm
    HostName EXTERNAL_IP
    User USER
    IdentityFile KEY_FILENAME directory
    LocalForward PORT_1 IP:PORT_1
    LocalForward PORT_2 IP:PORT_2
    LocalForward PORT_3 IP:PORT_3
```
  
    Example:

```bash
Host mlops-zoomcamp-vm
    HostName 34.125.197.156
    User w10
    IdentityFile C:\Users\w10\.ssh\gcp_ssh
    LocalForward 8888 localhost:8888
    LocalForward 5000 127.0.0.1:5000
    LocalForward 4200 0.0.0.0:4200
```


    The EXTERNAL_IP can change every time you power one the VM. 
Now you can type `ssh mlops-zoomcamp-vm` in your console and you'll get connected to the VM.

**Note0**: In step 4 in ```config``` file the last two lines are to forward multiple port through the same host, in this case 
```LocalForward 8888 localhost:8888``` is for jupyter, ```LocalForward 5000 127.0.0.1:5000``` is for MLflow and ```LocalForward 4200 0.0.0.0:4200``` is for Prefect. You can add more LocalForward if you want.

**Note1**: Don't forget to power off the VM after your work you can use ```sudo poweroff```. 

**Note2**: if you get the next warning:
```bash
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@    WARNING: REMOTE HOST IDENTIFICATION HAS CHANGED!     @
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
```
Then copy and paste the EXTERNAL_IP of your VM and type:

```bash
ssh-keygen -R "34.125.105.3"
```

[Back to the top](#)

### Install Dependencies <a name="dependencies"></a>

Now we need to install the next dependencies(you can update the links for Anaconda and Docker Compose):

- **Install Anaconda**:

```bash
cd ~
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
```

- **Install Docker**:

```bash
sudo apt update
sudo apt install docker.io
```

- **Install Docker Compose**

```bash
mkdir soft
cd soft/
wget https://github.com/docker/compose/releases/download/v2.5.0/docker-compose-linux-x86_64 -O docker-compose
chmod +x docker-compose
```

- **Modified PATH Varibles**


   Type ```nano .bashrc```, copy and paste the next at the end of the `.bashrc` file

```bash
export PATH="${HOME}/soft:${PATH}"
```

Type `source .bashrc`, now everything that is in `/soft` directory will be in the PATH then you can execute it everywhere.

- **Add current user to docker group**

```bash
sudo usermod -aG docker $USER
logout
```

Then logback to the VM.

- **Verify Installation**

```bash
which python
# /home/w10/anaconda3/bin/python

which docker
# /usr/bin/docker

which docker-compose
# /home/w10/soft/docker-compose

docker run hello-world
```

- **Run Jupyter Notebook**

```bash
jupyter notebook
```

[Back to the top](#)


## Course Overview <a name="overview"></a>

[Video source](https://www.youtube.com/watch?v=teP9KWkP6SM&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=6)

When data scientists experiment with Jupyter Notebooks for creating models, they often don't follow best practices and are often unstructured due to the nature of experimentation: cells are re-run with slightly different values and previous results may be lost, or the cell execution order may be inconsistent, for example.

***Module 2*** covers ***experiment tracking***: by using tools such as [MLflow](https://mlflow.org/) we will create ***experiment trackers*** (such as the history of cells that we've rerun multiple times) and ***model registries*** (for storing the models we've created during the experiments), instead of relying on our memory or janky setups such as external spreadsheets or convoluted naming schemes for our files.

***Module 3*** covers ***orchestration and ML pipelines***: by using tools such as [Prefect](https://www.prefect.io/) and [Kubeflow](https://www.kubeflow.org/) we can break down our notebooks into separate identifyable steps and connect them in order to create a ***ML pipeline*** which we can parametrize with the data and models we want and easily execute.

![asda](./Images/Module1/ML-pipeline.PNG)

***Module 4*** covers ***serving the models***: we will learn how to deploy models in different ways.

***Module 5*** covers ***model monitoring***: we will see how to check whether our model is performing fine or not and how to generate alers to warn us of performance drops and failures, and even automate retraining and redeploying models without human input.

***Module 6*** covers ***best practices***, such as how to properly maintain and package code, how to deploy successfully, etc.

***Module 7*** covers ***processes***: we will see how to properly communicate between all the stakeholders of a ML project (scientists, engineers, etc) and how to work together.

[Back to the top](#)


## MLOps Maturity Model <a name="maturity"></a>

[Video Source](https://www.youtube.com/watch?v=XwTH8BDGzYk&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=7)

[Table Source](https://docs.microsoft.com/en-us/azure/architecture/example-scenario/mlops/mlops-maturity-model)

A framework for classifying different levels of MLOps maturity is listed below:


| Lvl |              | Overview | Use Case | 
|----:|--------------|----------|----------|  
| 0️⃣  | **No MLOps** | <ul><li>ML process highly manual</li><li>poor cooperation</li><li>lack of standards, success depends on an individual's expertise</li> </ul> | <ul><li>proof of concept (PoC)</li><li>academic project</li></ul> |
| 1️⃣  | **DevOps but no MLOps** | <ul><li>ML training is most often manual </li><li>software engineers might help with the deployment</li><li>automated tests and releases</li> </ul> | <ul><li>bringing PoC to production</li></ul> |
| 2️⃣  | **Automated Training** | <ul><li>ML experiment results are centrally tracked </li><li>training code and models are version controlled</li><li>deployment is handled by software engineers</li> </ul> | <ul><li>maintaining 2-3+ ML models</li></ul> |
| 3️⃣  | **Automated Model Deployment** | <ul><li>releases are managed by an automated CI/CD pipeline</li><li>close cooperation between data and software engineers</li><li>performance of the deployed model is monitored, A/B tests for model selection are used</li></ul> | <ul><li>business-critical ML services</li></ul> |
| 4️⃣  | **Full MLOps Automated Operations** | <ul><li>clearly defined metrics for model monitoring</li><li>automatic retraining triggered when passing a model metric's threshold</li> </ul>  | <ul><li>use only when a favorable trade-off between implementation cost and increase in efficiency is likely</li><li>retraining is needed often and is repetitive (has potential for automation)</li></ul> |

Be aware that not every project or even every part of a project needs to have the highest maturity level possible because it could exceed the project's resource budget. **Pragmatism is key**.


[Back to the top](#)

# Module 3: Orchestration and ML Pipelines <a name="module3"></a>

## Negative engineering and workflow orchestration <a name="negative-and-orchestration"></a>

[Video Source](https://www.youtube.com/watch?v=eKzCjNXoCTc&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=20)

**Workflow Orchestration**

It's a set of tools that schedule and monitor work that you want to accomplish. Ex: Scheduling ML models training

Exmaple pipeline: 
```
PostgresQL -> Parquet -> Pandas -> Sklearn -> mlflow
                                      ↳ Rest API ↳ Flask (If deploying)
```
Random Points of Failure can occur in the pipeline. The goal of the workflow orchestration is to minimize the errors and fail
gracefully.

In more interconnected pipelines (Different pipelines interconnected) failure points are more common.

**Negative Engineering**

90% of engineering time is spent on:
+ Retries when APIs go down
+ Malformed data
+ Notifications
+ Observability into Failure
+ Conditional Failure Logic
+ Timeouts

Prefect's goal is to reduce this time to increase productivity; The goal is to reduce the time spent on Negative Engineering.

[Back to the top](#)

## Introducting Prefect <a name="prefect"></a>

[Video Source](https://www.youtube.com/watch?v=Yb6NJwI7bXw&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=21)

Open Source Workflow Orchestration Framework for eliminating Negative Engineering:
+ Open Source
+ Python-based
+ Modern data stack
+ Native Dask integration
+ Very active Community
+ Prefect Cloud/Prefect Server -> Cloud is hosted by Prefect, Server Self-hosted.

Prefect Orion (aka Prefect 2.0) is an overwhole on Prefect 1.0; No backwards/forwards compatibility. **It's in Beta**

**Current state of Prefect**:

+ Prefect Core (1.0)
+ Prefect Orion (2.0 beta)
+ Prefect uses Decorators to wrap the code.

**Deploying Notebooks**:

We don't deploy notebooks, and if deployed they're deployed as a single step. Notebooks are thus refactored into scripts for
deployment

[Back to the top](#)







## First Prefect flow and basics <a name="prefect-flow"></a>

[Video Source](https://www.youtube.com/watch?v=MCFpURG506w&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=22)

### Adding a  Prefect flow <a name="flow"></a>

We implement Prefect in our code by wrapping the workflow function (which fetches the data, preprocesses it, trains the model...etc) with a `@flow` decorator as:

```python
from prefect import flow

@flow
def main():
  ...
```
This enables extra logging. 

The main function is what is usually put in a `if "__name"" == "__main__":` bloc wrapped as a function. (Name doesn't matter)

Multiple Flows can be put in the same file.

### Adding a Prefect task <a name="task"></a>

(Note: Prefect 1.0/2.0 difference: In 2.0 we can mix and match between normal functions and `@task` functions.
In 1.0 we couldn't)

We can then add tasks by using the `@task` decorator around our task function (example: preprocessing, training... etc):
```python
@task
def train_model(X,y):
  ...
```
The output of a function wrapped around a `@task` is a `PrefectFuture` object. If we're mixing and matching normal functions 
with `@task`ed functions we need to get the result of the function by calling `.result()` on the `PrefectFuture`. For example:
```python
from prefect import flow, task

X_train, X_val, y_train, y_val, dv = add_features(train_path, val_path).result()
```
Otherwise we'll just get the `PrefectFuture` object (likely with a crash).

Adding a task enables further logging.

Tasks can also have parameters like caching and retries.

### Sequential Task Running <a name="seq"></a>
If tasks don't depend on each other, Prefect will run the tasks asynchronously by default. This can't be handled by MLflow, so
in order to force sequential task running we add a parameter to the `@flow` decorator:

```python
from prefect import flow, task
from prefect.task_runners import SequentialTaskRunner
@flow(task_runner=SequentialTaskRunner())
def main():
  ...
```

### Logs <a name="logs"></a>

If you have prints inside a task you could'nt see in the Prefect UI, you should remove the prints and use ```get_run_logger``` instead.

```python
from prefect import flow, task, get_run_logger

@task
def train_model(X, y):
    logger = get_run_logger()
    logger.info("My log")
    ...
```

### Prefect UI <a name="ui"></a>

We can open the Prefect UI (In Orion) using `prefect orion start` to spin up a localhost instance.
The UI contains information and logs of varying detail about each flow run and where errors may have occured in the process
as well as error Stacktraces and task flows.

### Parameter Type Validation <a name="val"></a>
If an Orion flow receives a bad parameter type, instead of running the flow and inevitably failing, it will instead not run the flow at all and output a failed run to save compute time.

[Back to the top](#)













## Remote Prefect Orion Deployment <a name="remote"></a>

[Video Source](https://www.youtube.com/watch?v=ComkSIAB0k4&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=23)

### Remote Prefect Orion<a name="orion"></a>

In order to remotely deploy Prefect. The remote VM needs some ports open:

| Connection Type | Port  |
|-----------------|-------|
| HTTP            | [80]  |
| HTTPS           | [443] |
| TCP             | 4200  |
| UDP             | 4200  |

[How to do it in AWS](https://vanchiv.com/open-port-on-aws-ec2-instance/)
[How to do it in GCP](https://www.howtogeek.com/devops/how-to-open-firewall-ports-on-a-gcp-compute-engine-instance/)

(Source could be set as "Anywhere" for AWS or "0.0.0.0/0" for GCP)

To start Prefect Orion, [we could follow these steps by Kevin](https://discourse.prefect.io/t/hosting-an-orion-instance-on-a-cloud-vm/967):

+ `pip install prefect==2.0b5` (Replace with most recent 2.0 version on pip)
+ Set the PREFECT_ORION_UI_API_URL with :
`prefect config set PREFECT_ORION_UI_API_URL="http://<external-ip>:4200/api"`
+ Start Orion:
`prefect orion start --host 0.0.0.0`
+ On the local machine, configure the API:
`prefect config set PREFECT_API_URL="http://<external-ip>:4200/api"`
+ The remote UI will be visible on port 4200. Just type ```http://<external-ip>:4200```.  Example:  http://34.125.197.156:4200/


You should see the variables being set with `prefect config view`.

In case the prefect UI_API_URL or PREFECT_API_URL is already set to an older IP address, we can unset the variable by:
```bash
prefect config unset PREFECT_ORION_UI_API_URL
```
and then set the key as described above. (Replace PREFECT_ORION_UI_API_URL with PREFECT_API_URL for resetting PREFECT_API_URL)

Now when running a script with Prefect Flow, the data should be logged into the remote VM Prefect instance.

### Using Prefect Cloud <a name="cloud"></a>

Instead of running Prefect on a VM ourselves, we could use Prefect's Cloud service at https://beta.prefect.io which provides 
token login in addition to all other Prefect features.

### Defining Storage for Prefect localy: <a name="storage-localy"></a>

In order to view the Prefect current configures storage we use `prefect storage ls`. By default, Prefect has no storage set and it stores results for runs in a temporary directory in its runtime environment. In order to create storage, we use `prefect storage create` then selecting the storage type we want if you choose ```Local Storage``` then you'll need to provide a path to storage the files, for example ```/home/username/.prefect```.

### Defining Storage for Prefect in AWS S3: <a name="storage-aws"></a>

**Creating and Configuring AWS S3 Storage**:

Free Tier Note: AWS S3 has 5GB free storage for free with somewhat limited read(GET) and more limited write(PUT) ops.

First create an S3 Bucket; Search for S3 in the "Services" search bar and select "S3" (NOT "S3 Glacier").

![S3](Images/Module3/S3.png)

**Adding a User with S3 Permissions to AWS**:

In order to access the S3 Bucket with Prefect, we need to add a new user with S3 Permissions:

1 - Adding a User: To create the "User" which Prefect will use to access the S3 Bucket, we open the drop-down menu next to the account nameat the top right and select "Security Credentials" > "Users" (Left menu) > "Add Users".

![Security Credentials](Images/Module3/security_credentials.png)

![Users](Images/Module3/users.png)

2 - Add a new user (ex: prefect) with an AWS Access Type of "Programmatic Access" and select "Next:Permissions".

![Programmatic Access](Images/Module3/programmaticaccess.png)

3 - We want to add a group with S3 Permissions and add our user to it; Select "Create Group" and name it (ex: S3-FullAccess) then in the policies search for "S3FullAccess" and select it (If you click on it, it redirects you to the policy details and you'll have to start over).

![S3FullAccess](Images/Module3/S3FullAccess.png)

4 - Select the new group and click "Next:Tags" (Put tags here if you want), then "Next:Review" then "Create User". Don't close this window! The Access Secret Key is irretrievable once lost!

![Access Keys](Images/Module3/accesskeys.png)

5 - Create new Prefect storage with `prefect storage create` and select Amazon S3.

6 - Give the name of the S3 Bucket you've created beforehand.

7 - Copy the access key from the User Window and paste it when prompted "AWS ACCESS KEY ID".

8 - Copy the secret access key from the User Window and paste it when prompted "AWS SECRET ACCESS KEY".

9 - Skip SESSION TOKEN, PROFILE NAME, REGION NAME (Press Enter).

10 - Choose a "Locally" unique name for the configuration.

11 - When prompted if you want to set it as default, select Y.

Optional: Store the credentials in a CSV file.

[Back to the top](#)













## Deployment of Prefect flow <a name="deploy-prefect"></a>

[Video Source](https://www.youtube.com/watch?v=xw9JfaWPPps&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=24)

### Deployment <a name="deploy"></a>

To store the run results, we have to modify our flow file; First we import:

```python
from prefect.deployments import DeploymentSpec
from prefect.orion.schemas.schedules import IntervalSchedule
from prefect.flow_runners import SubprocessFlowRunner
from datetime import timedelta
```
Note that `SubprocessFlowRunner` is for non-containerized runs, if using Kubernetes or Docker we use something different.

We then define a `DeploymentSpec`:
```python
DeploymentSpec(
  flow=main,
  name="momdel_training",
  schedule=IntervalSchedule(interval=timedelta(minutes=5)),
  flow_runner=SubprocessFlowRunner(),
  tags=["ml"]
)
```
`flow` is the flow to run. (Note `main` is no longer called explicitly)

`schedule` is the schedule at which we run the flow. For example here every 5 minutes we run `main`.

`tags` are tags associated with the flow. They can be used for filtering for example.

`flow_runner` in this case specifies that the flow will only be ran locally; i.e: Not on Kubernetes or Docker containers.

To create a deployment, we use:
`prefect deployment create prefect_deploy.py` 

this only creates the deployment and schedules the runs. It does not know how to run them. To run them, we use Work Queues.


### CronSchedule <a name="schedule"></a>
if you want to schedule to run for a certain day of a month you can't do that with ```IntervalSchedule``` you should use ```CronSchedule``` instead, example:


```python
from prefect.deployments import DeploymentSpec
from prefect.orion.schemas.schedules import CronSchedule

DeploymentSpec(
    name="cron-schedule-deployment",
    flow_location="/path/to/flow.py",
    schedule=CronSchedule(
        cron="0 0 * * *",
        timezone="America/New_York"),
)

```

[Source](https://orion-docs.prefect.io/concepts/schedules/#cronschedule)

you need to provide a cron string, with the next sintax:

```bash
┌───────────── minute (0 - 59)
│ ┌───────────── hour (0 - 23)
│ │ ┌───────────── day of the month (1 - 31)
│ │ │ ┌───────────── month (1 - 12)
│ │ │ │ ┌───────────── day of the week (0 - 6) (Sunday to Saturday;
│ │ │ │ │                                   7 is also Sunday on some systems)
│ │ │ │ │
│ │ │ │ │
* * * * * <command to execute>
```

Example Cron expression to run a flow at 9 AM every 15th of the month?

```python 
"0 9 15 * *"
``` 

[Source](https://en.wikipedia.org/wiki/Cron)


### Work Queues <a name="queue"></a>:

The work queue a queue that will prompt its attached Agents to run the scheduled runs.

To create a new work queue use the Prefect UI and select work queues in the side panel (A name is required for creating the queue), filtering by Tags is possible. A window will pop that includes the command used to add an agent to the work-queue; `prefect agent start <UUID>`. With the UUID being the UUID of the work-queue.

We can check the state of the work-queue by using `prefect work-queue preview <UUID>` where it will show scheduled runs. 

#### Adding a Local Agent <a name="agent"></a>

Note: If the storage used is local, and the Agent is ran elsewhere it won't be able to get the files necessary for the run.

The Agent is what runs the scheduled runs for the work-queue. It checks every 5 seconds whether there is work to do in the work-queue, fetches the flow file from storage and runs it.

To run the agent we use the provided command on the local computer in the work-queue page:
`prefect agent start <UUID>`

[Back to the top](#)















## References <a name="references"></a>

https://github.com/ziritrion/mlopszoomcamp/blob/main/notes/1_intro.md

https://github.com/LoHertel/Road-to-MLOps/blob/main/01-primer/README.md

https://gist.github.com/Qfl3x/8dd69b8173f027b9468016c118f3b6a5


https://cloud.google.com/compute/docs/connect/create-ssh-keys

https://cloud.google.com/compute/docs/connect/add-ssh-keys



















# Para las notas